In [48]:
import tensorflow as tf
import numpy as np
#Download the word embeddings from: http://nlp.stanford.edu/data/glove.6B.zip
#load embedding file
embedding_dim=100
path_to_glove_file = "embeddings/glove.6B/glove.6B.100d.txt"



In [169]:
embeddings_index = {}
game_vocab_list=[]
game_vocab_len=10000
with open(path_to_glove_file) as f:
    i=0
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
        if i<game_vocab_len:
            game_vocab_list.append(word)
        i+=1

print("Found %s word vectors." % len(embeddings_index))
print("created game_vocab_list using", len(game_vocab_list), "words.")


Found 400000 word vectors.
created game_vocab_list using 10000 words.


In [170]:
#remove stopwords when creating vocab list
# test_vocab_list=['apple', 'orange', 'tree', 'machine', 'rotor', 'elephant', 'jump', 'run', 'laugh', 'misery','glass']
vocab_list=game_vocab_list
search_matrix = np.zeros((len(vocab_list), embedding_dim))
hits=0
misses=0
for i,word in enumerate(vocab_list):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        search_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
        

        
print(hits, misses, np.shape(search_matrix))

10000 0 (10000, 100)


In [296]:
print(embeddings_matrix)
#lower case
#check if in embeddings_index
def euclidean_distances(entered_word):
    #entered_word can be any word in the embedding, not just the test_vocab_list
    distances=np.sqrt(np.sum((search_matrix-embeddings_index[entered_word])**2,axis=1))
    return distances

def cosine_similarity(entered_word):
    #entered_word can be any word in the embedding, not just the test_vocab_list
    numerator=np.dot(search_matrix,embeddings_index[entered_word])
    denominator=np.sqrt(np.linalg.norm(search_matrix,axis=1))*np.sqrt(np.linalg.norm(embeddings_index[entered_word]))
    return numerator/denominator

def custom_distance(entered_word):
    return euclidean_distances(entered_word)-cosine_similarity(entered_word)

check_word='canine'
# print(euclidean_distances(check_word))
print(vocab_list[np.argmin(euclidean_distances(check_word))])

# print(cosine_similarity(check_word))
print(vocab_list[np.argmax(cosine_similarity(check_word))])
# print(cosine_similarity(check_word)/euclidean_distances(check_word))
print(vocab_list[np.argmin(custom_distance(check_word))])



[]
dog
dog
dog


In [292]:
#let's use euclidean distance, because that's easiest to see how it will work with 3 points.

#given 2 words, find the word closest to both of them

word1='toy'
word2='tree'

def combined_distances(word1,word2):
    distance_to_w1=euclidean_distances(word1)
    distance_to_w2=euclidean_distances(word2)
    combined_distances=np.sqrt(distance_to_w1**2+distance_to_w2**2)
    #can't pick the word1 or word2. Easier to set to large value than delete so I don't mess up indicies.
    combined_distances[np.argwhere(distance_to_w1==0)]+=10**10
    combined_distances[np.argwhere(distance_to_w2==0)]+=10**10
    return combined_distances

def middle_word(word1,word2):
    distances=combined_distances(word1,word2)
    return(game_vocab_list[np.argmin(distances)])

def distance_to_word(check_word, word1,word2):
    distances=combined_distances(word1,word2)
    index = game_vocab_list.index(check_word)
    return(distances[index])

print(distance_to_word('baseball', word1, word2))
print(middle_word(word1,word2))

# game_vocab_list[np.argmin(combined_distances)]

10.033321043872665
cat
